In [ ]:
# phase 2
import pandas as pd
import numpy as np

# Assume you have Stage 1 combined top predictions
# combined_top_df: top X rows per company per timeframe
# target_stage1: "PredictedTomorrowClose"
# target_stage2: "TomorrowReturn" (to predict in Stage 2)
stage2_features = [c for c in combined_top_df.select_dtypes(include=[np.number]).columns 
                   if c not in [target_stage1, target_stage2]]

# -------------------------
# Option 1: SARIMAX (per company)
# -------------------------
from statsmodels.tsa.statespace.sarimax import SARIMAX

top_companies = combined_top_df['CompanyId'].unique()
sarimax_results = []

for cid in top_companies:
    df_c = combined_top_df[combined_top_df['CompanyId'] == cid].copy()
    ts = df_c[target_stage2].dropna()
    
    if len(ts) < 10:  # skip very short histories
        continue
    
    # Fit SARIMAX (example order; tune per dataset)
    model = SARIMAX(ts, order=(1,0,1), seasonal_order=(1,1,1,5), enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    
    # Predict next step
    pred = res.get_forecast(steps=1).predicted_mean.iloc[0]
    sarimax_results.append({
        'CompanyId': cid,
        'Pred_TomorrowReturn_SARIMAX': pred
    })

sarimax_df = pd.DataFrame(sarimax_results)
print("SARIMAX predictions:")
print(sarimax_df.head())

# -------------------------
# Option 2: PyCaret Automated ML
# -------------------------
# Note: install pycaret: pip install pycaret
from pycaret.regression import setup, compare_models, predict_model

# Fill NaNs for PyCaret
combined_top_df_clean = combined_top_df.fillna(0)

# Setup PyCaret
regression_setup = setup(
    data=combined_top_df_clean,
    target=target_stage2,
    numeric_features=stage2_features,
    silent=True,
    session_id=42
)

# Automatically compare multiple regression models
best_model = compare_models()

# Predict Stage 2 target for all top companies
pycaret_predictions = predict_model(best_model, data=combined_top_df_clean)
combined_top_df_clean["Pred_TomorrowReturn_PyCaret"] = pycaret_predictions["Label"]

print("PyCaret predictions:")
print(combined_top_df_clean[['CompanyId', 'Pred_TomorrowReturn_PyCaret']].head())

# -------------------------
# Combine results for comparison
# -------------------------
final_comparison_df = combined_top_df_clean.merge(
    sarimax_df,
    on="CompanyId",
    how="left"
)[['CompanyId', 'Pred_TomorrowReturn_SARIMAX', 'Pred_TomorrowReturn_PyCaret']]

print("Combined Stage 2 predictions (SARIMAX vs PyCaret):")
print(final_comparison_df.head())


top_n = 5  # number of top companies to select per timeframe

# Merge Stage 2 predictions back into the combined_top_df
combined_top_df_clean = combined_top_df_clean.merge(
    sarimax_df,
    on="CompanyId",
    how="left"
)

# Add PyCaret predictions (already in combined_top_df_clean)
combined_top_df_clean["Pred_TomorrowReturn_PyCaret"] = pycaret_predictions["Label"]

# Function to select top N per timeframe based on a given prediction column
def select_top_n(df, pred_col, n=5):
    top_list = []
    for tf in df['TimeFrame'].unique():
        tf_df = df[df['TimeFrame'] == tf].copy()
        # Sort descending by prediction
        tf_df = tf_df.sort_values(pred_col, ascending=False)
        # Take top N per timeframe
        top_list.append(tf_df.head(n))
    return pd.concat(top_list, ignore_index=True)

# Top N companies per timeframe for SARIMAX
top_sarimax_df = select_top_n(combined_top_df_clean, 'Pred_TomorrowReturn_SARIMAX', top_n)
print("Top N companies per timeframe based on SARIMAX predictions:")
print(top_sarimax_df[['CompanyId', 'TimeFrame', 'Pred_TomorrowReturn_SARIMAX']])

# Top N companies per timeframe for PyCaret
top_pycaret_df = select_top_n(combined_top_df_clean, 'Pred_TomorrowReturn_PyCaret', top_n)
print("Top N companies per timeframe based on PyCaret predictions:")
print(top_pycaret_df[['CompanyId', 'TimeFrame', 'Pred_TomorrowReturn_PyCaret']])

# Optional: Combine both rankings for comparison
top_combined_df = top_sarimax_df.merge(
    top_pycaret_df,
    on=['CompanyId', 'TimeFrame'],
    how='outer',
    suffixes=('_SARIMAX', '_PyCaret')
)
print("Combined top N rankings (SARIMAX vs PyCaret):")
print(top_combined_df)


In [1]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from pyspark.sql.functions import lit, current_timestamp
import pandas as pd
import numpy as np
from pyspark.sql.types import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import display, HTML
from pyspark.sql import functions as F
from pyspark.sql.window import Window


spark = init_spark("bsf_candidates_analysis", log_level="WARN", show_progress=False, enable_ui=True)
engine = init_mariadb_engine()

ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 200)         # Adjust width for readability
pd.set_option("display.max_rows", 20)       # Show only top 20 rows by default

# Show tables
tables_df = spark.sql("SHOW TABLES IN bsf")
tables_df.show(truncate=False)

# Add row count for each table
for row in tables_df.collect():
    table_name = row['tableName']
    full_name = f"bsf.{table_name}"
    
    try:
        count = spark.table(full_name).count()
    except Exception as e:
        count = f"Error: {e}"
    
    print(f"Table: {full_name} | Rows: {count}")


df_last = spark.table("bsf.daily_signals_last_allcol ")
df_all = spark.table("bsf.daily_signals")

df_all.groupBy("Action").count().orderBy(F.desc("count")).show(truncate=False)

print(df_all.groupBy("TimeFrame", "Action") \
  .count() \
  .orderBy("TimeFrame", F.desc("count")) \
  .show(truncate=False))

:: loading settings :: url = jar:file:/home/jupyter/.venv/python3.9_bsf/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b78611e-c7e2-4877-be7a-6607d50c44ab;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0rc1 in spark-list
	found io.delta#delta-storage;3.0.0rc1 in spark-list
	found org.antlr#antlr4-runtime;4.9.3 in spark-list
:: resolution report :: resolve 614ms :: artifacts dl 47ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0rc1 from spark-list in [default]
	io.delta#delta-storage;3.0.0rc1 from spark-list in [default]
	org.antlr#antlr4-runtime;4.9.3 from spark-list in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	

[Spark] Started 'bsf_candidates_analysis' log_level=WARN (effective=WARN), progress=False


25/09/09 16:55:34 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/09 16:55:39 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/09 16:55:39 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/09 16:55:39 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/09 16:55:40 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/09 16:55:40 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/09 16:55:40 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/09 16:55:44 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/09 16:55:44 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/09 16:55:44 WARN HiveConf: HiveConf of name hive.stats.retries.w

+---------+-------------------------+-----------+
|namespace|tableName                |isTemporary|
+---------+-------------------------+-----------+
|bsf      |company                  |false      |
|bsf      |companystockhistory      |false      |
|bsf      |daily_signals            |false      |
|bsf      |daily_signals_allcol     |false      |
|bsf      |daily_signals_last       |false      |
|bsf      |daily_signals_last_allcol|false      |
+---------+-------------------------+-----------+

Table: bsf.company | Rows: 30949
Table: bsf.companystockhistory | Rows: 459664
Table: bsf.daily_signals | Rows: 1837828
Table: bsf.daily_signals_allcol | Rows: 1837828
Table: bsf.daily_signals_last | Rows: 7372
Table: bsf.daily_signals_last_allcol | Rows: 7372
+------+-------+
|Action|count  |
+------+-------+
|Hold  |1304016|
|Buy   |296025 |
|Sell  |237787 |
+------+-------+

+---------+------+------+
|TimeFrame|Action|count |
+---------+------+------+
|Daily    |Hold  |322337|
|Daily    |Buy

In [2]:
df = df_last.cache()
# -----------------------------
# Aggregate Buy/Sell/Hold counts per company per timeframe
# -----------------------------
df_counts = df.groupBy("CompanyId", "TimeFrame").agg(
    F.sum(F.when(F.col("Action") == "Buy", 1).otherwise(0)).alias("BuyCount"),
    F.sum(F.when(F.col("Action") == "Sell", 1).otherwise(0)).alias("SellCount"),
    F.sum(F.when(F.col("Action") == "Hold", 1).otherwise(0)).alias("HoldCount"),
    F.sum("Return").alias("Return")
)

# -----------------------------
# Define window partitioned by timeframe
# -----------------------------
w_buy = Window.partitionBy("TimeFrame").orderBy(F.desc("BuyCount"))
w_sell = Window.partitionBy("TimeFrame").orderBy(F.desc("SellCount"))
w_hold = Window.partitionBy("TimeFrame").orderBy(F.desc("HoldCount"))

# -----------------------------
# Add separate rank columns
# -----------------------------
df_ranked = (
    df_counts
    .withColumn("BuyRank", F.row_number().over(w_buy))
    .withColumn("SellRank", F.row_number().over(w_sell))
    .withColumn("HoldRank", F.row_number().over(w_hold))
)

# -----------------------------
# Select what you want
# -----------------------------
ranked_companies = df_ranked.select(
    "CompanyId", "TimeFrame", "BuyCount", "SellCount", "HoldCount", 
    "Return", "BuyRank", "SellRank", "HoldRank"
)

# -----------------------------
# Join back to the original df to get full rows with rank by last return
# -----------------------------
ranked_rows = df.join(ranked_companies, on=["CompanyId", "TimeFrame"], how="inner")

display(ranked_rows.filter(F.col("BuyRank") <= 1).orderBy("TimeFrame", "BuyRank").toPandas())


,CompanyId,TimeFrame,StockDate,Open,High,Low,Close,Doji,Hammer,HangingMan,InvertedHammer,ShootingStar,BullishMarubozu,BearishMarubozu,SuspiciousCandle,BullishEngulfing,BearishEngulfing,BullishHarami,BearishHarami,HaramiCross,PiercingLine,DarkCloudCover,MorningStar,EveningStar,ThreeWhiteSoldiers,ThreeBlackCrows,TweezerTop,TweezerBottom,InsideBar,OutsideBar,NearHigh,NearLow,PatternCount,PatternType,MA,MA_slope,UpTrend_MA,DownTrend_MA,RecentReturn,UpTrend_Return,DownTrend_Return,Volatility,LowVolatility,HighVolatility,ROC,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,ValidHammer,ValidBullishEngulfing,ValidPiercingLine,ValidMorningStar,ValidThreeWhiteSoldiers,ValidBullishMarubozu,ValidTweezerBottom,ValidShootingStar,ValidBearishEngulfing,ValidDarkCloud,ValidEveningStar,ValidThreeBlackCrows,ValidBearishMarubozu,ValidTweezerTop,ValidHaramiCross,ValidBullishHarami,ValidBearishHarami,ValidInsideBar,ValidOutsideBar,ValidCustomSignal1,ValidCustomSignal2,TomorrowClose,TomorrowReturn,Return,AvgReturn,MomentumZ,BuyThresh,SellThresh,MomentumAction,BullScore,BearScore,PatternScore,PatternScoreNorm,PatternAction,CandleAction,CandidateAction,Action,TomorrowAction,TomorrowActionSource,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,ValidAction,HasValidSignal,SignalStrength,BatchId,IngestedAt,BuyCount,SellCount,HoldCount,Return,BuyRank,SellRank,HoldRank
0,41647,Daily,2025-09-05,0.01400,0.0171,0.0140,0.01610,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,3,ThreeWhiteSoldiers,0.015433,-0.037003,False,True,0.035370,True,False,0.078226,False,True,0.150000,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.035370,-0.002773,0.487598,0.019031,-0.034301,Buy,1.0,1.0,0.0,0.0,Hold,Buy,Buy,Buy,Hold,LastRowHold,0.333406,0.333406,0.333406,0.083406,194.0,True,True,2,41647_20250909_184644,20250909_184644,1,0,0,0.035370,1,232,1555
1,68742,Long,2025-09-05,0.06125,0.0682,0.0531,0.06095,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,3,ThreeWhiteSoldiers,0.056263,0.097023,True,False,0.128704,True,False,0.097298,False,True,0.128704,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.033051,0.016289,0.172270,0.035717,-0.039594,Buy,26.0,1.0,25.0,2.5,Buy,Hold,Buy,Buy,Hold,LastRowHold,0.791972,0.791972,0.791972,0.791972,146.0,True,True,1,68742_20250909_181627,20250909_181627,1,0,0,0.033051,1,220,1552
2,35067,Short,2025-09-05,0.00510,0.0100,0.0051,0.01000,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,3,ThreeWhiteSoldiers,0.008000,0.951220,True,False,0.666667,True,False,0.351599,False,True,0.923077,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.666667,0.120263,1.554054,0.002738,-0.017071,Buy,7.0,1.0,6.0,2.0,Buy,Hold,Buy,Buy,Hold,LastRowHold,0.656971,0.656971,0.656971,0.656971,160.0,True,True,1,35067_20250909_170002,20250909_170002,1,0,0,0.666667,1,254,1572
3,30613,Swing,2025-09-05,0.05330,0.0658,0.0533,0.06050,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,3,ThreeWhiteSoldiers,0.049622,0.110907,True,False,0.344444,True,False,0.100250,False,True,0.344444,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.163462,0.039425,1.237281,0.020165,-0.017107,Buy,7.0,5.0,2.0,0.4,Buy,Hold,Buy,Buy,Hold,LastRow

In [3]:
# -----------------------------

# Convert to Pandas and save as csv
# -----------------------------
#List of timeframes
timeframes = ["Short", "Swing", "Long", "Daily"]

for tf in timeframes:
    df_name = f"pdf_{tf.lower()}"  # e.g., "pdf_short"
    df_name_all = f"pdf_{tf.lower()}_all"  # e.g., "pdf_short_all"
    globals()[df_name] = ranked_rows.filter(F.col("TimeFrame") == tf).toPandas()
    globals()[df_name_all] = df_all.filter(F.col("TimeFrame") == tf).toPandas()
    #ranked_rows.filter(F.col("TimeFrame") == tf).toPandas().to_csv(f"cvs/{tf.lower()}_output.csv", index=False)
    
# pdf_short = ranked_rows.filter(F.col("TimeFrame") == "Short").toPandas()
# pdf_short.to_csv(f"cvs/short_output.csv", index=False)

timeframe_dfs = {
    "Short": pdf_short,
    "Swing": pdf_swing,
    "Long": pdf_long,
    "Daily": pdf_daily
}
timeframe_dfs_all = {
    "Short": pdf_short_all,
    "Swing": pdf_swing_all,
    "Long": pdf_long_all,
    "Daily": pdf_daily_all
}

In [13]:
timeframes = ["Short", "Swing", "Long", "Daily"]

# Store Spark DFs only (no toPandas here)
timeframe_dfs = {tf: ranked_rows.filter(F.col("TimeFrame") == tf) for tf in timeframes}
timeframe_dfs_all = {tf: df_all.filter(F.col("TimeFrame") == tf) for tf in timeframes}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

target = "TomorrowClose"

all_predictions = []  # to collect per-timeframe predictions
epsilon = 1e-6        # for log transformations if needed

for tf, sdf_tf in timeframe_dfs_all.items():
    print(f"\n=== Processing timeframe: {tf} ===")

    # Convert only this timeframe to Pandas
    pdf_tf = sdf_tf.toPandas()
    # Copy so you don’t mutate original
    df_tf = pdf_tf.copy()
    
    # Compute correlation with target   
    numeric_cols = df_tf.select_dtypes(include=[np.number]).columns.tolist()
    if target in numeric_cols:
        numeric_cols.remove(target) # duplicate column causes error in corr statement
    corr = df_tf[numeric_cols + [target]].corr()[target].sort_values(ascending=False)
    #print(corr)
    # Filter features with "high enough" correlation (absolute value)
    threshold = 0.03  # example threshold
    good_features = corr.index.drop(target)[corr.abs().loc[corr.index.drop(target)] >= threshold].tolist()
    
    #print("Selected ML features based on correlation:", good_features)


    # Filter for rows with valid target for training
    train_mask = pdf_tf[target].notna()
    train_df = pdf_tf[train_mask].copy()
    
    if train_df.empty:
        print(f"⚠️ No historical data for {tf}, skipping.")
        continue

    # Optional: log-transform numeric features
    for col in ["Open", "High", "Low", "Close"]:
        train_df[f"log_{col}"] = np.log(train_df[col].replace(0, epsilon))
    
    # Training features and target
    X_train = train_df[good_features].fillna(0)
    y_train = train_df[target]

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    print(f"R² score (train): {model.score(X_train, y_train):.4f}")

    # Predict on historical for residuals (optional)
    train_df["PredictedTomorrowClose"] = model.predict(X_train)
    train_df["PredictedReturn"] = (train_df["PredictedTomorrowClose"] - train_df["Close"]) / train_df["Close"]

    # Predict for future rows
    future_mask = pdf_tf[target].isna()
    future_df = pdf_tf[future_mask].copy()
    
    if not future_df.empty:
        X_future = future_df[good_features].fillna(0)
        future_df["PredictedTomorrowClose"] = model.predict(X_future)
        future_df["PredictedReturn"] = (future_df["PredictedTomorrowClose"] - future_df["Close"]) / future_df["Close"]
        future_df["TimeFrame"] = tf
        
        # Keep only valid predictions
        valid_preds = future_df[future_df["PredictedTomorrowClose"].notna()]
        all_predictions.append(valid_preds)

# Combine predictions from all timeframes
combined_df = pd.concat(all_predictions, ignore_index=True)

In [25]:
# Sort by predicted return
combined_df.sort_values("PredictedReturn", ascending=False, inplace=True)
#display(combined_df[cols_to_show].head(20))

# Sort within each timeframe by predicted return
top_per_timeframe = (
    combined_df
    .sort_values(["TimeFrame", "PredictedReturn"], ascending=[True, False])
    .groupby("TimeFrame")
    .head(3)  # top 3 per timeframe
)

In [26]:
display(top_per_timeframe[cols_to_show])

,CompanyId,TimeFrame,StockDate,Open,Close,TomorrowAction,PredictedTomorrowClose,PredictedReturn,Return
7329,103592,Daily,2025-09-05,0.0100,0.0110,Hold,0.013202,0.200205,109.000000
5869,33524,Daily,2025-08-29,0.0115,0.0115,Hold,0.011558,0.005001,56.500000
6866,249416,Daily,2025-09-05,0.0100,0.0100,Hold,0.011963,0.196345,32.333333
5483,103592,Long,2025-09-05,0.0100,0.0110,Hold,0.012110,0.100902,109.000000
4006,33524,Long,2025-08-29,0.0115,0.0115,Hold,0.011962,0.040169,56.500000
5119,249416,Long,2025-09-05,0.0100,0.0100,Hold,0.012109,0.210947,32.333333
1798,103592,Short,2025-09-05,0.0100,0.0110,Hold,0.013548,0.231595,109.000000
329,33524,Short,2025-08-29,0.0115,0.0115,Hold,0.011434,-0.005764,56.500000
1312,249416,Short,2025-09-05,0.0100,0.0100,Hold,0.011764,0.176389,32.333333
3646,103592,Swing,2025-09-05,0.0100,0.0110,Hold,0.011967,0.087892,109.000000


In [ ]:
#new by company 

In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

target = "TomorrowClose"
epsilon = 1e-6
all_predictions = []

# Loop through each timeframe
for tf, sdf_tf in timeframe_dfs_all.items():
    print(f"\n=== Processing timeframe: {tf} ===")
    
    # Convert Spark DF to Pandas for this timeframe
    pdf_tf = sdf_tf.toPandas()
    
    # Get list of companies
    companies = pdf_tf['CompanyId'].unique()
    
    for cid in companies:
        df_company = pdf_tf[pdf_tf['CompanyId'] == cid].copy()
        # Add log columns to all rows (train + future)
        for col in ["Open", "High", "Low", "Close"]:
            df_company[f"log_{col}"] = np.log(df_company[col].replace(0, epsilon))
    
        # Skip if no valid target
        train_mask = df_company[target].notna()
        train_df = df_company[train_mask].copy()
        if train_df.empty:
            continue
        

        # Compute correlation-based features for this company
        numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
        if target in numeric_cols:
            numeric_cols.remove(target)
        corr = train_df[numeric_cols + [target]].corr()[target].sort_values(ascending=False)
        threshold = 0.03
        good_features = corr.index[corr.abs() >= threshold].tolist()
        
        # Training
        X_train = train_df[good_features].fillna(0)
        y_train = train_df[target]
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        #print(f"Company {cid} | R² score (train): {model.score(X_train, y_train):.4f}")
        
        # Predict future
        future_mask = df_company[target].isna()
        future_df = df_company[future_mask].copy()
        if not future_df.empty:
            X_future = future_df[good_features].fillna(0)
            future_df["PredictedTomorrowClose"] = model.predict(X_future)
            future_df["PredictedReturn"] = (future_df["PredictedTomorrowClose"] - future_df["Close"]) / future_df["Close"]
            future_df["TimeFrame"] = tf
            all_predictions.append(future_df)

# Combine all predictions
combined_df = pd.concat(all_predictions, ignore_index=True)
combined_df.sort_values("PredictedReturn", ascending=False, inplace=True)



=== Processing timeframe: Short ===
Company 58954 | R² score (train): 1.0000
Company 30344 | R² score (train): 1.0000
Company 73662 | R² score (train): 1.0000
Company 67853 | R² score (train): 1.0000
Company 88907 | R² score (train): 1.0000
Company 41691 | R² score (train): 1.0000
Company 65149 | R² score (train): 1.0000
Company 59011 | R² score (train): 1.0000
Company 98069 | R² score (train): 1.0000
Company 30301 | R² score (train): 1.0000
Company 98033 | R² score (train): 1.0000
Company 34497 | R² score (train): 1.0000
Company 55859 | R² score (train): 1.0000
Company 2090 | R² score (train): 1.0000
Company 30413 | R² score (train): 1.0000
Company 78882 | R² score (train): 1.0000
Company 1163 | R² score (train): 1.0000
Company 41316 | R² score (train): 1.0000
Company 59504 | R² score (train): 1.0000
Company 44596 | R² score (train): 1.0000
Company 657 | R² score (train): 1.0000
Company 64080 | R² score (train): 1.0000
Company 38907 | R² score (train): 1.0000
Company 98115 | R² score

In [ ]:
# phase 2
import pandas as pd
import numpy as np

# Assume you have Stage 1 combined top predictions
# combined_top_df: top X rows per company per timeframe
# target_stage1: "PredictedTomorrowClose"
# target_stage2: "TomorrowReturn" (to predict in Stage 2)
stage2_features = [c for c in combined_top_df.select_dtypes(include=[np.number]).columns 
                   if c not in [target_stage1, target_stage2]]

# -------------------------
# Option 1: SARIMAX (per company)
# -------------------------
from statsmodels.tsa.statespace.sarimax import SARIMAX

top_companies = combined_top_df['CompanyId'].unique()
sarimax_results = []

for cid in top_companies:
    df_c = combined_top_df[combined_top_df['CompanyId'] == cid].copy()
    ts = df_c[target_stage2].dropna()
    
    if len(ts) < 10:  # skip very short histories
        continue
    
    # Fit SARIMAX (example order; tune per dataset)
    model = SARIMAX(ts, order=(1,0,1), seasonal_order=(1,1,1,5), enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    
    # Predict next step
    pred = res.get_forecast(steps=1).predicted_mean.iloc[0]
    sarimax_results.append({
        'CompanyId': cid,
        'Pred_TomorrowReturn_SARIMAX': pred
    })

sarimax_df = pd.DataFrame(sarimax_results)
print("SARIMAX predictions:")
print(sarimax_df.head())

# -------------------------
# Option 2: PyCaret Automated ML
# -------------------------
# Note: install pycaret: pip install pycaret
from pycaret.regression import setup, compare_models, predict_model

# Fill NaNs for PyCaret
combined_top_df_clean = combined_top_df.fillna(0)

# Setup PyCaret
regression_setup = setup(
    data=combined_top_df_clean,
    target=target_stage2,
    numeric_features=stage2_features,
    silent=True,
    session_id=42
)

# Automatically compare multiple regression models
best_model = compare_models()

# Predict Stage 2 target for all top companies
pycaret_predictions = predict_model(best_model, data=combined_top_df_clean)
combined_top_df_clean["Pred_TomorrowReturn_PyCaret"] = pycaret_predictions["Label"]

print("PyCaret predictions:")
print(combined_top_df_clean[['CompanyId', 'Pred_TomorrowReturn_PyCaret']].head())

# -------------------------
# Combine results for comparison
# -------------------------
final_comparison_df = combined_top_df_clean.merge(
    sarimax_df,
    on="CompanyId",
    how="left"
)[['CompanyId', 'Pred_TomorrowReturn_SARIMAX', 'Pred_TomorrowReturn_PyCaret']]

print("Combined Stage 2 predictions (SARIMAX vs PyCaret):")
print(final_comparison_df.head())


top_n = 5  # number of top companies to select per timeframe

# Merge Stage 2 predictions back into the combined_top_df
combined_top_df_clean = combined_top_df_clean.merge(
    sarimax_df,
    on="CompanyId",
    how="left"
)

# Add PyCaret predictions (already in combined_top_df_clean)
combined_top_df_clean["Pred_TomorrowReturn_PyCaret"] = pycaret_predictions["Label"]

# Function to select top N per timeframe based on a given prediction column
def select_top_n(df, pred_col, n=5):
    top_list = []
    for tf in df['TimeFrame'].unique():
        tf_df = df[df['TimeFrame'] == tf].copy()
        # Sort descending by prediction
        tf_df = tf_df.sort_values(pred_col, ascending=False)
        # Take top N per timeframe
        top_list.append(tf_df.head(n))
    return pd.concat(top_list, ignore_index=True)

# Top N companies per timeframe for SARIMAX
top_sarimax_df = select_top_n(combined_top_df_clean, 'Pred_TomorrowReturn_SARIMAX', top_n)
print("Top N companies per timeframe based on SARIMAX predictions:")
print(top_sarimax_df[['CompanyId', 'TimeFrame', 'Pred_TomorrowReturn_SARIMAX']])

# Top N companies per timeframe for PyCaret
top_pycaret_df = select_top_n(combined_top_df_clean, 'Pred_TomorrowReturn_PyCaret', top_n)
print("Top N companies per timeframe based on PyCaret predictions:")
print(top_pycaret_df[['CompanyId', 'TimeFrame', 'Pred_TomorrowReturn_PyCaret']])

# Optional: Combine both rankings for comparison
top_combined_df = top_sarimax_df.merge(
    top_pycaret_df,
    on=['CompanyId', 'TimeFrame'],
    how='outer',
    suffixes=('_SARIMAX', '_PyCaret')
)
print("Combined top N rankings (SARIMAX vs PyCaret):")
print(top_combined_df)

In [ ]:
pdf_tf[target].describe()

import pandas as pd

all_top_gainers = []  # List to collect DataFrames

for tf, pdf in timeframe_dfs.items():
    #print(f"\n=== Processing timeframe: {tf} ===")
    
    pdf_tf = pdf.copy()
    
    # Log-transform features
    for col in ["Open", "High", "Low", "Close"]:
        pdf_tf[f"log_{col}"] = np.log(pdf_tf[col].replace(0, epsilon))
    pdf_tf["log_TomorrowClose"] = np.log(pdf_tf["TomorrowClose"].replace(0, epsilon))
    
    # Historical rows
    hist_mask = pdf_tf["TomorrowClose"].notna()
    model_df = pdf_tf[hist_mask].copy()
    if model_df.empty:
        print("⚠️ No historical data, skipping timeframe.")
        continue

    X = model_df[features].fillna(0)
    y = model_df["log_TomorrowClose"]

    # Train model
    ols = LinearRegression()
    ols.fit(X, y)

    # Predict for latest rows
    future_mask = pdf_tf["TomorrowClose"].isna()
    latest_rows = (
        pdf_tf[future_mask]
        .sort_values("StockDate")
        .groupby("CompanyId", as_index=False)
        .tail(1)
    )
    if not latest_rows.empty:
        latest_X = latest_rows[features].fillna(0)
        latest_rows["PredictedTomorrowClose"] = np.exp(ols.predict(latest_X))
        latest_rows["PredictedReturn"] = (
            latest_rows["PredictedTomorrowClose"] - latest_rows["Close"]
        ) / latest_rows["Close"]
        latest_rows["TimeFrame"] = tf  # Add timeframe column
        
        # Keep only valid predictions
        valid_preds = latest_rows[latest_rows["PredictedTomorrowClose"].notna()]
        all_top_gainers.append(valid_preds)

# Concatenate all timeframes
combined_df = pd.concat(all_top_gainers, ignore_index=True)

# Display top 20 by PredictedReturn across all timeframes
combined_df.sort_values("PredictedReturn", ascending=False, inplace=True)

# Sort by PredictedReturn descending (highest return first)
top_candidates = combined_df.sort_values("PredictedReturn", ascending=False)

# Display top 20
display(top_candidates.head(10))
# Define the columns you want to see
cols_to_show = ["CompanyId", "TimeFrame", "StockDate", "Open", "Close", "TomorrowAction", "PredictedTomorrowClose", "Return", "Rank"]

# Show only those columns from top_candidates
display(top_candidates[cols_to_show].head(10))


In [ ]:
top_candidates.to_csv(f"cvs/top_candidates_output.csv", index=False)

In [ ]:
filtered_gainers = top_gainers[
    (top_gainers["PredictedReturn"] > 0.05) &
    (top_gainers["TomorrowAction"] == "Buy") &
    (top_gainers["ActionConfidence"] > 0.5)
]
display(filtered_gainers)

In [ ]:
spark.stop()

In [29]:
# the newest
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge

# Settings
target = "TomorrowClose"
epsilon = 1e-6
lasso_alpha = 0.01
ridge_alpha = 1.0
corr_threshold = 0.03  # minimum correlation to keep feature

all_predictions = []
all_results = []

# Loop through each timeframe
for tf, sdf_tf in timeframe_dfs_all.items():
    print(f"\n=== Processing timeframe: {tf} ===")
    
    # Convert Spark DF to Pandas for this timeframe
    pdf_tf = sdf_tf.toPandas()
    companies = pdf_tf['CompanyId'].unique()
    
    for cid in companies:
        df_company = pdf_tf[pdf_tf['CompanyId'] == cid].copy()
        
        # Add log columns for numeric prices
        for col in ["Open", "High", "Low", "Close"]:
            df_company[f"log_{col}"] = np.log(df_company[col].replace(0, epsilon))
        
        # Filter rows with valid target
        train_mask = df_company[target].notna()
        train_df = df_company[train_mask].copy()
        if train_df.empty:
            continue
        
        # Correlation-based feature selection
        numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
        if target in numeric_cols:
            numeric_cols.remove(target)
        corr = train_df[numeric_cols + [target]].corr()[target].sort_values(ascending=False)
        good_features = corr.index[corr.abs() >= corr_threshold].tolist()
        
        if not good_features:
            continue
        
        X_train = train_df[good_features].fillna(0)
        y_train = train_df[target]
        
        # --- Linear Regression ---
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)
        lr_r2 = lr_model.score(X_train, y_train)
        
        # --- Lasso ---
        lasso_model = Lasso(alpha=lasso_alpha, max_iter=10000)
        lasso_model.fit(X_train, y_train)
        lasso_r2 = lasso_model.score(X_train, y_train)
        lasso_features = [f for f, coef in zip(good_features, lasso_model.coef_) if abs(coef) > 1e-6]
        
        # --- Ridge ---
        ridge_model = Ridge(alpha=ridge_alpha)
        ridge_model.fit(X_train, y_train)
        ridge_r2 = ridge_model.score(X_train, y_train)
        ridge_features = [f for f, coef in zip(good_features, ridge_model.coef_) if abs(coef) > 1e-6]
        
        # Store model results
        all_results.append({
            "TimeFrame": tf,
            "CompanyId": cid,
            "LR_R2": lr_r2,
            "Lasso_R2": lasso_r2,
            "Ridge_R2": ridge_r2,
            "LR_Features": good_features,
            "Lasso_Features": lasso_features,
            "Ridge_Features": ridge_features
        })
        
        # --- Predict future ---
        future_mask = df_company[target].isna()
        future_df = df_company[future_mask].copy()
        if not future_df.empty:
            X_future = future_df[good_features].fillna(0)
            
            # Predictions for each model
            future_df["LR_PredictedTomorrowClose"] = lr_model.predict(X_future)
            future_df["LR_PredictedReturn"] = (future_df["LR_PredictedTomorrowClose"] - future_df["Close"]) / future_df["Close"]
            
            future_df["Lasso_PredictedTomorrowClose"] = lasso_model.predict(X_future)
            future_df["Lasso_PredictedReturn"] = (future_df["Lasso_PredictedTomorrowClose"] - future_df["Close"]) / future_df["Close"]
            
            future_df["Ridge_PredictedTomorrowClose"] = ridge_model.predict(X_future)
            future_df["Ridge_PredictedReturn"] = (future_df["Ridge_PredictedTomorrowClose"] - future_df["Close"]) / future_df["Close"]
            
            future_df["TimeFrame"] = tf
            all_predictions.append(future_df)

# Combine all predictions
combined_df = pd.concat(all_predictions, ignore_index=True)
combined_df.sort_values("LR_PredictedReturn", ascending=False, inplace=True)

# Combine all model results into a DataFrame
results_df = pd.DataFrame(all_results)

print("\n=== Model Comparison Results ===")
display(results_df.head(10))

print("\n=== Top Predictions ===")
cols_to_show = [
    "CompanyId", "TimeFrame", "StockDate", "Open", "Close",
    "LR_PredictedTomorrowClose", "LR_PredictedReturn",
    "Lasso_PredictedTomorrowClose", "Lasso_PredictedReturn",
    "Ridge_PredictedTomorrowClose", "Ridge_PredictedReturn"
]
display(combined_df[cols_to_show].head(20))



=== Processing timeframe: Short ===


TypeError: solve() got an unexpected keyword argument 'sym_pos'

In [ ]:
# another new 2 stage:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge

target_stage1 = "TomorrowClose"   # Stage 1 prediction target
target_stage2 = "TomorrowReturn"  # Stage 2 target for final ML
epsilon = 1e-6
top_x = 5  # top companies/signals per timeframe

all_top_predictions = []  # collect top X per company per timeframe

# Loop through each timeframe
for tf, sdf_tf in timeframe_dfs_all.items():
    print(f"\n=== Processing timeframe: {tf} ===")
    
    pdf_tf = sdf_tf.toPandas()
    companies = pdf_tf['CompanyId'].unique()
    
    for cid in companies:
        df_company = pdf_tf[pdf_tf['CompanyId'] == cid].copy()
        
        # Add log features
        for col in ["Open", "High", "Low", "Close"]:
            df_company[f"log_{col}"] = np.log(df_company[col].replace(0, epsilon))
        
        # Filter training rows
        train_mask = df_company[target_stage1].notna()
        train_df = df_company[train_mask].copy()
        if train_df.empty:
            continue
        
        # Numeric features
        numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
        if target_stage1 in numeric_cols:
            numeric_cols.remove(target_stage1)
        
        # Stage 1 ML models
        X_train = train_df[numeric_cols].fillna(0)
        y_train = train_df[target_stage1]

        # Train multiple models
        models = {
            "LinearRegression": LinearRegression(),
            "Lasso": Lasso(alpha=0.001),
            "Ridge": Ridge(alpha=0.01, solver='svd')
        }
        preds_stage1 = {}
        
        for name, model in models.items():
            model.fit(X_train, y_train)
            train_score = model.score(X_train, y_train)
            train_df[f"Pred_{name}"] = model.predict(X_train)
            preds_stage1[name] = train_df[f"Pred_{name}"]
            print(f"Company {cid} | {name} R²: {train_score:.4f}")
        
        # Rank top X by predicted return (using LinearRegression as main example)
        train_df["PredReturn"] = (train_df["Pred_LinearRegression"] - train_df["Close"]) / train_df["Close"]
        top_rows = train_df.nlargest(top_x, "PredReturn").copy()
        top_rows["TimeFrame"] = tf
        all_top_predictions.append(top_rows)

# Combine all top X selections
combined_top_df = pd.concat(all_top_predictions, ignore_index=True)

# Now combined_top_df is ready for Stage 2 ML:
# Use it to predict another target, e.g., TomorrowReturn
stage2_features = [c for c in combined_top_df.select_dtypes(include=[np.number]).columns 
                   if c not in [target_stage1, target_stage2]]
X_stage2 = combined_top_df[stage2_features].fillna(0)
y_stage2 = combined_top_df[target_stage2]

# Example: Stage 2 ML using LinearRegression
final_model = LinearRegression()
final_model.fit(X_stage2, y_stage2)
combined_top_df["Pred_TomorrowReturn"] = final_model.predict(X_stage2)

# Sort by predicted return for final selection
combined_top_df.sort_values("Pred_TomorrowReturn", ascending=False, inplace=True)

# Display top rows
cols_to_show = ["CompanyId", "TimeFrame", "StockDate", "Close", "Pred_LinearRegression", 
                "Pred_Lasso", "Pred_Ridge", "PredReturn", "Pred_TomorrowReturn"]
display(combined_top_df[cols_to_show].head(20))



=== Processing timeframe: Short ===
Company 58954 | LinearRegression R²: 0.9967
Company 58954 | Lasso R²: 0.9194
Company 58954 | Ridge R²: 0.9956
Company 30344 | LinearRegression R²: 0.9968
Company 30344 | Lasso R²: 0.8451
Company 30344 | Ridge R²: 0.9934
Company 73662 | LinearRegression R²: 0.9969
Company 73662 | Lasso R²: 0.9127
Company 73662 | Ridge R²: 0.9949
Company 67853 | LinearRegression R²: 0.9919
Company 67853 | Lasso R²: 0.7893
Company 67853 | Ridge R²: 0.9823
Company 88907 | LinearRegression R²: 0.9907
Company 88907 | Lasso R²: 0.9268
Company 88907 | Ridge R²: 0.9901
Company 41691 | LinearRegression R²: 0.9897
Company 41691 | Lasso R²: 0.8740
Company 41691 | Ridge R²: 0.9891
Company 65149 | LinearRegression R²: 0.9965
Company 65149 | Lasso R²: 0.8917
Company 65149 | Ridge R²: 0.9942
Company 59011 | LinearRegression R²: 0.9973
Company 59011 | Lasso R²: 0.8262
Company 59011 | Ridge R²: 0.9948
Company 98069 | LinearRegression R²: 0.9959
Company 98069 | Lasso R²: 0.7819
Compan

/home/jupyter/.venv/python3.9_bsf/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.422e-03, tolerance: 2.111e-03
  model = cd_fast.enet_coordinate_descent(


Company 55859 | Ridge R²: 0.9877
Company 2090 | LinearRegression R²: 0.9911
Company 2090 | Lasso R²: 0.8308
Company 2090 | Ridge R²: 0.9885
Company 30413 | LinearRegression R²: 0.9893
Company 30413 | Lasso R²: 0.8247
Company 30413 | Ridge R²: 0.9879
Company 78882 | LinearRegression R²: 0.9983
Company 78882 | Lasso R²: 0.6112
Company 78882 | Ridge R²: 0.9947
Company 1163 | LinearRegression R²: 0.9856
Company 1163 | Lasso R²: 0.8060
Company 1163 | Ridge R²: 0.9823
Company 41316 | LinearRegression R²: 0.9909
Company 41316 | Lasso R²: 0.8956
Company 41316 | Ridge R²: 0.9890
Company 59504 | LinearRegression R²: 0.9960
Company 59504 | Lasso R²: 0.9049
Company 59504 | Ridge R²: 0.9905
Company 44596 | LinearRegression R²: 0.9242
Company 44596 | Lasso R²: 0.9047
Company 44596 | Ridge R²: 0.9231
Company 657 | LinearRegression R²: 0.9968
Company 657 | Lasso R²: 0.9128
Company 657 | Ridge R²: 0.9959
Company 64080 | LinearRegression R²: 0.9959
Company 64080 | Lasso R²: 0.7362
Company 64080 | Ridge 

/home/jupyter/.venv/python3.9_bsf/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.162e-02, tolerance: 4.183e-03
  model = cd_fast.enet_coordinate_descent(


Company 67997 | Ridge R²: 0.9822
Company 30237 | LinearRegression R²: 0.9954
Company 30237 | Lasso R²: 0.8692
Company 30237 | Ridge R²: 0.9921
Company 65093 | LinearRegression R²: 0.9885
Company 65093 | Lasso R²: 0.9137
Company 65093 | Ridge R²: 0.9856
Company 41631 | LinearRegression R²: 0.9902
Company 41631 | Lasso R²: 0.7816
Company 41631 | Ridge R²: 0.9759
Company 34380 | LinearRegression R²: 0.9742
Company 34380 | Lasso R²: 0.8373
Company 34380 | Ridge R²: 0.9722
Company 34749 | LinearRegression R²: 0.9904
Company 34749 | Lasso R²: 0.7931
Company 34749 | Ridge R²: 0.9875
Company 44109 | LinearRegression R²: 0.9968
Company 44109 | Lasso R²: 0.8532
Company 44109 | Ridge R²: 0.9919
Company 65460 | LinearRegression R²: 0.9959
Company 65460 | Lasso R²: 0.9956
Company 65460 | Ridge R²: 0.9959


...

Company 89124 | LinearRegression R²: 0.9830
Company 89124 | Lasso R²: 0.8379
Company 89124 | Ridge R²: 0.9634
Company 77649 | LinearRegression R²: 0.9785
Company 77649 | Lasso R²: 0.3177
Company 77649 | Ridge R²: 0.9704
Company 100591 | LinearRegression R²: 0.9932
Company 100591 | Lasso R²: 0.3586
Company 100591 | Ridge R²: 0.9846
Company 61883 | LinearRegression R²: 0.9751
Company 61883 | Lasso R²: 0.6013
Company 61883 | Ridge R²: 0.9660
Company 67846 | LinearRegression R²: 0.8991
Company 67846 | Lasso R²: 0.3837
Company 67846 | Ridge R²: 0.8805
Company 65792 | LinearRegression R²: 0.9838
Company 65792 | Lasso R²: 0.7616
Company 65792 | Ridge R²: 0.9781
Company 42278 | LinearRegression R²: 0.9891
Company 42278 | Lasso R²: 0.8907
Company 42278 | Ridge R²: 0.9853
Company 84945 | LinearRegression R²: 0.9769
Company 84945 | Lasso R²: 0.6521
Company 84945 | Ridge R²: 0.9715
Company 100628 | LinearRegression R²: 0.9839
Company 100628 | Lasso R²: 0.9063
Company 100628 | Ridge R²: 0.9826
Comp

/home/jupyter/.venv/python3.9_bsf/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.018e-01, tolerance: 8.631e-02
  model = cd_fast.enet_coordinate_descent(


Company 68117 | LinearRegression R²: 0.9794
Company 68117 | Lasso R²: 0.1918
Company 68117 | Ridge R²: 0.9752
Company 65366 | LinearRegression R²: 0.9932
Company 65366 | Lasso R²: 0.8795
Company 65366 | Ridge R²: 0.9898
Company 1849 | LinearRegression R²: 0.9953
Company 1849 | Lasso R²: 0.4180
Company 1849 | Ridge R²: 0.9943
Company 61876 | LinearRegression R²: 0.9930
Company 61876 | Lasso R²: 0.9222
Company 61876 | Ridge R²: 0.9918
Company 89455 | LinearRegression R²: 0.9773
Company 89455 | Lasso R²: 0.8318
Company 89455 | Ridge R²: 0.9653
Company 41713 | LinearRegression R²: 0.9896
Company 41713 | Lasso R²: 0.9395
Company 41713 | Ridge R²: 0.9866
Company 93376 | LinearRegression R²: 0.9944
Company 93376 | Lasso R²: 0.8535
Company 93376 | Ridge R²: 0.9933
Company 65805 | LinearRegression R²: 0.9759
Company 65805 | Lasso R²: 0.7958
Company 65805 | Ridge R²: 0.9583
Company 100182 | LinearRegression R²: 0.9743
Company 100182 | Lasso R²: 0.0849
Company 100182 | Ridge R²: 0.9713
Company 78

/home/jupyter/.venv/python3.9_bsf/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.073e-05, tolerance: 4.996e-05
  model = cd_fast.enet_coordinate_descent(


Company 53383 | LinearRegression R²: 0.9795
Company 53383 | Lasso R²: 0.8189
Company 53383 | Ridge R²: 0.9742
Company 78038 | LinearRegression R²: 0.9782
Company 78038 | Lasso R²: 0.1751
Company 78038 | Ridge R²: 0.9638
Company 88865 | LinearRegression R²: 0.9956
Company 88865 | Lasso R²: 0.8865
Company 88865 | Ridge R²: 0.9868
Company 59490 | LinearRegression R²: 0.9842
Company 59490 | Lasso R²: 0.4528
Company 59490 | Ridge R²: 0.9839
Company 58410 | LinearRegression R²: 0.9798
Company 58410 | Lasso R²: 0.9032
Company 58410 | Ridge R²: 0.9787
Company 41965 | LinearRegression R²: 0.9963
Company 41965 | Lasso R²: 0.7929
Company 41965 | Ridge R²: 0.9889
Company 44569 | LinearRegression R²: 0.9927
Company 44569 | Lasso R²: 0.3467
Company 44569 | Ridge R²: 0.9748
Company 75930 | LinearRegression R²: 0.9797
Company 75930 | Lasso R²: 0.3031
Company 75930 | Ridge R²: 0.9766
Company 76209 | LinearRegression R²: 0.9580
Company 76209 | Lasso R²: 0.7154
Company 76209 | Ridge R²: 0.9482
Company 23

In [ ]:
# phase 2
import pandas as pd
import numpy as np

# Assume you have Stage 1 combined top predictions
# combined_top_df: top X rows per company per timeframe
# target_stage1: "PredictedTomorrowClose"
# target_stage2: "TomorrowReturn" (to predict in Stage 2)
stage2_features = [c for c in combined_top_df.select_dtypes(include=[np.number]).columns 
                   if c not in [target_stage1, target_stage2]]

# -------------------------
# Option 1: SARIMAX (per company)
# -------------------------
from statsmodels.tsa.statespace.sarimax import SARIMAX

top_companies = combined_top_df['CompanyId'].unique()
sarimax_results = []

for cid in top_companies:
    df_c = combined_top_df[combined_top_df['CompanyId'] == cid].copy()
    ts = df_c[target_stage2].dropna()
    
    if len(ts) < 10:  # skip very short histories
        continue
    
    # Fit SARIMAX (example order; tune per dataset)
    model = SARIMAX(ts, order=(1,0,1), seasonal_order=(1,1,1,5), enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    
    # Predict next step
    pred = res.get_forecast(steps=1).predicted_mean.iloc[0]
    sarimax_results.append({
        'CompanyId': cid,
        'Pred_TomorrowReturn_SARIMAX': pred
    })

sarimax_df = pd.DataFrame(sarimax_results)
print("SARIMAX predictions:")
print(sarimax_df.head())

# -------------------------
# Option 2: PyCaret Automated ML
# -------------------------
# Note: install pycaret: pip install pycaret
from pycaret.regression import setup, compare_models, predict_model

# Fill NaNs for PyCaret
combined_top_df_clean = combined_top_df.fillna(0)

# Setup PyCaret
regression_setup = setup(
    data=combined_top_df_clean,
    target=target_stage2,
    numeric_features=stage2_features,
    silent=True,
    session_id=42
)

# Automatically compare multiple regression models
best_model = compare_models()

# Predict Stage 2 target for all top companies
pycaret_predictions = predict_model(best_model, data=combined_top_df_clean)
combined_top_df_clean["Pred_TomorrowReturn_PyCaret"] = pycaret_predictions["Label"]

print("PyCaret predictions:")
print(combined_top_df_clean[['CompanyId', 'Pred_TomorrowReturn_PyCaret']].head())

# -------------------------
# Combine results for comparison
# -------------------------
final_comparison_df = combined_top_df_clean.merge(
    sarimax_df,
    on="CompanyId",
    how="left"
)[['CompanyId', 'Pred_TomorrowReturn_SARIMAX', 'Pred_TomorrowReturn_PyCaret']]

print("Combined Stage 2 predictions (SARIMAX vs PyCaret):")
print(final_comparison_df.head())


top_n = 5  # number of top companies to select per timeframe

# Merge Stage 2 predictions back into the combined_top_df
combined_top_df_clean = combined_top_df_clean.merge(
    sarimax_df,
    on="CompanyId",
    how="left"
)

# Add PyCaret predictions (already in combined_top_df_clean)
combined_top_df_clean["Pred_TomorrowReturn_PyCaret"] = pycaret_predictions["Label"]

# Function to select top N per timeframe based on a given prediction column
def select_top_n(df, pred_col, n=5):
    top_list = []
    for tf in df['TimeFrame'].unique():
        tf_df = df[df['TimeFrame'] == tf].copy()
        # Sort descending by prediction
        tf_df = tf_df.sort_values(pred_col, ascending=False)
        # Take top N per timeframe
        top_list.append(tf_df.head(n))
    return pd.concat(top_list, ignore_index=True)

# Top N companies per timeframe for SARIMAX
top_sarimax_df = select_top_n(combined_top_df_clean, 'Pred_TomorrowReturn_SARIMAX', top_n)
print("Top N companies per timeframe based on SARIMAX predictions:")
print(top_sarimax_df[['CompanyId', 'TimeFrame', 'Pred_TomorrowReturn_SARIMAX']])

# Top N companies per timeframe for PyCaret
top_pycaret_df = select_top_n(combined_top_df_clean, 'Pred_TomorrowReturn_PyCaret', top_n)
print("Top N companies per timeframe based on PyCaret predictions:")
print(top_pycaret_df[['CompanyId', 'TimeFrame', 'Pred_TomorrowReturn_PyCaret']])

# Optional: Combine both rankings for comparison
top_combined_df = top_sarimax_df.merge(
    top_pycaret_df,
    on=['CompanyId', 'TimeFrame'],
    how='outer',
    suffixes=('_SARIMAX', '_PyCaret')
)
print("Combined top N rankings (SARIMAX vs PyCaret):")
print(top_combined_df)